# Audio Sample to Sega Genesis Synth Path Estimator - Algorithm 7

In [2]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential # Will experiment with various architectures
from tensorflow.keras.layers import Dense

# Side note: don't actually run this in Jupyter. This is simply a whiteboard for the model design.
# Transfer to a .py file, and run this on Linux with AMD GPU support.

2023-02-15 18:53:58.143941: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


SETUP

In [5]:
#open directory with the spectrograms, PCA-reduced audio data (if I used it), and register data in it.
used_raw_data = False

os.chdir('..')
os.chdir('./data/')

RegisterTargets = pd.read_csv('SampleRegisters.csv', index_col = 0).T
RegisterTargets = RegisterTargets.loc[RegisterTargets.Algorithm == 7] # filter by algorithm
RegisterTargets = RegisterTargets.drop(['Algorithm', 'Ratio1', 'Attack1', 'Decay1', 'Sustain1',
                        'Ratio2', 'Attack2', 'Decay2', 'Sustain2',
                        'Ratio3', 'Attack3', 'Decay3', 'Sustain3',
                        'Ratio4', 'Attack4', 'Decay4', 'Sustain4',
                        'ReleaseLv1', 'ReleaseLv2', 'ReleaseLv3', 'ReleaseLv4', ], axis = 'columns') # Not needed here.
print(len(RegisterTargets),'patches created with algorithm 7 found.')

SpectrumFeatures = pd.read_csv('SampleSpectra.csv', index_col = [0,1]).T.loc[RegisterTargets.index]
print(SpectrumFeatures.head())

print('Register data for patches:')
print(RegisterTargets.head())

if used_raw_data:
    WavFeatures = pd.read_csv('SampleWav.csv', index_col = 0).loc[RegisterTargets.index]
    SpectrumFeatures = SpectrumFeatures.join(WavFeatures)

pca_spect = PCA(n_components = numComponents)
pca_spect.fit(SpectrumFeatures)
reduced = pca_spect.fit_transform(SpectrumFeatures)
print('Original shape ', np.shape(SpectrumFeatures))
print('Transformed shape ', np.shape(reduced))
print('Components')
print(pca_spect.components_)
print('Explained variance')
print(pca_spect.explained_variance_)

plt.plot(np.cumsum(pca_spect.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');





232 patches created with algorithm 7 found.


KeyError: "['07 - Marble Garden Zone Act 1_817.wav'] not in index"

TRAINING AN ESTIMATOR MODEL

In [3]:
model7 = Sequential()
model7.add(Dense(250, input_shape = (129,629), activation='relu'))
model7.add(Dense(200, activation='relu'))
model7.add(Dense(160, activation='relu'))
model7.add(Dense(140, activation='relu'))
model7.add(Dense(120, activation='relu'))
model7.add(Dense(100, activation='relu'))
model7.add(Dense(80, activation='relu'))
model7.add(Dense(60, activation='relu'))
model7.add(Dense(37, activation='relu')) # 41 numerical outputs, but only 37 if we exclude release.

# To do: to decide on these options.
model7.compile(loss = '?', optimizer = 'adam', metrics = ['?'])

# To do: to wrangle the data to fit into X, y format.
model7.fit(X,y,epochs = 150, batch_size = 10)

_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

[-6.13680145e+00  2.12331397e+01  1.12052298e+01 -8.47762943e+00
 -1.60953251e+01 -2.72292366e+01 -3.07423799e+01 -3.15654074e+01
 -2.70938598e+01 -3.03187354e+01 -2.80391258e+01 -3.23866879e+01
 -3.57411729e+01 -3.78762370e+01 -3.92524898e+01 -4.10550396e+01
 -4.21225654e+01 -4.54594141e+01 -4.33877405e+01 -4.41984576e+01
  4.61154619e+04  5.36191007e+04 -5.23138112e+02  2.29247090e+04
  2.66074887e+04 -2.96259108e+03  1.98822613e+04 -8.13982487e+03
 -3.01927284e+03  1.61927192e+04 -1.97006031e+04 -8.22352835e+03
 -8.85503055e+03  4.11727483e+02  4.07796951e+03 -5.04914564e+03
  1.77838870e+03 -1.48229187e+03 -2.04124798e+03  1.32328612e+03
 -8.99750566e+03 -1.80739857e+03  1.53847285e+03 -4.04891449e+03
 -3.23221750e+03 -5.75909638e+03  2.43062403e+03  3.30298931e+03
 -1.37364956e+03  1.91845194e+03 -4.75923948e+03  2.17952461e+03
  1.65974853e+02 -1.20860465e+03  4.87163410e+02 -7.85418732e+03
 -4.67340720e+03  3.40457916e+03  4.39521629e+03  1.96409254e+03
  4.87283897e+03  2.86040

MLPRegressor(alpha=1e-05, hidden_layer_sizes=(50, 50), max_iter=10000,
             random_state=1, solver='lbfgs')

SAVING THE MODEL TO JSON TO BE IMPORTED INTO MAIN PROGRAM

In [5]:
os.chdir('..')
os.chdir('./data/')
SpectrumFeatures = pd.read_csv('SampleSpectra.csv', index_col = [0,1]).T#.loc[RegisterTargets.index]
print(SpectrumFeatures.head())

Frequency                         0                                       \
Time                              0           1          2           3     
01 - Circle of Life_0.wav  479.603330  286.281700  16.643503  389.145400   
01 - Circle of Life_1.wav    3.184877    8.141680  22.113195   36.346890   
01 - Circle of Life_2.wav    0.572485    0.018849   0.020290    0.771428   
01 - Circle of Life_3.wav    0.108402    2.121624  12.437200    0.001558   
01 - Circle of Life_4.wav    0.284686   23.930990  11.773583    5.364444   

Frequency                                                                \
Time                             4           5           6          7     
01 - Circle of Life_0.wav  20.132044   56.910778  208.063220   3.752028   
01 - Circle of Life_1.wav   9.413405   25.061806   38.335815   5.365879   
01 - Circle of Life_2.wav   0.946703    0.007717    0.037849   0.038244   
01 - Circle of Life_3.wav  54.423275  128.358140    5.108642  77.306520   
01 - Circle of Li